In [7]:
import boto3
import pandas as pd
import time
import io

                 

GLUE_DATABASE = 'implementationdb'
bucket = 'sagemaker-us-east-1-646862220717'
ATHENA_S3_OUTPUT_PATH = 's3://sagemaker-us-east-1-646862220717/athena_results'

athena_query = '''SELECT * FROM reseller'''


def athena_read(athena_query):
    # Start query execution using Athena client
    athena_client = boto3.client('athena')
    response = athena_client.start_query_execution(
        QueryString=athena_query,
        QueryExecutionContext={
            'Database': GLUE_DATABASE
        },
        ResultConfiguration={
            'OutputLocation': ATHENA_S3_OUTPUT_PATH
        }
    )

    # Get Query Execution Id from above response
    query_execution_id = response['QueryExecutionId']
    print('Query Execution ID: ' + query_execution_id)

    # Poll for status of the query execution until it is succeeded, failed or cancelled
    query_execution_status = None
    RETRY_COUNT = 150 # Calculate based on poll time and max time taken by your queries
    query_status_check_count = 1
    while query_execution_status == 'QUEUED' or query_execution_status == 'RUNNING' or query_execution_status is None:
        # Get Query Execution
        query_status = athena_client.get_query_execution(
            QueryExecutionId=query_execution_id)
        query_execution_status = query_status['QueryExecution']['Status']['State']
        print('Query Status: ' + query_execution_status)
        if query_execution_status == 'FAILED' or query_execution_status == 'CANCELLED':
            raise Exception('Athena query with the string "{}" failed or was cancelled'.format(athena_query))

        if query_status_check_count <= RETRY_COUNT:
            query_status_check_count = query_status_check_count+1

            # Add a sleep time of 5 seconds between polls to check the status of query
            time.sleep(5)
        else:
            athena_client.stop_query_execution(QueryExecutionId = query_execution_id)
            raise Exception('Time execeeded based on the retry_count set. Aborting the query execution.')

    print('Query "{}" finished.'.format(athena_query))

    # Since S3 is eventually consistent, sometimes it takes a few seconds time for file to reflect back in S3. Hence adding sleep for safeside.
    time.sleep(3)

    # Get Query Results into a pandas dataframe
    athena_results_filepath = ATHENA_S3_OUTPUT_PATH + '/' + query_execution_id + '.csv'
    print('Athena Results File Path: '+ athena_results_filepath)
    
    athena_key = 'athena_results' + '/' + query_execution_id + '.csv'
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=athena_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    return df

In [8]:
athena_read(athena_query)

Query Execution ID: 2bafe8fc-33dd-4c73-8fff-710c2f1da1f6
Query Status: QUEUED
Query Status: SUCCEEDED
Query "SELECT * FROM reseller" finished.
Athena Results File Path: s3://sagemaker-us-east-1-646862220717/athena_results/2bafe8fc-33dd-4c73-8fff-710c2f1da1f6.csv


,id_reseller,zone,cluster
0,499929090,1019,A
1,499929094,1019,A
2,499929097,1019,A
3,499929098,1019,A
4,499929163,1014,A
...,...,...,...
12353,500783410,1091,B
12354,500784679,1090,D
12355,500787138,1091,B
12356,500792168,1091,C


In [9]:
ses

NameError: name 'ses' is not defined